In [1]:
import numpy as np

In [2]:
#creating the player and dealer
class Player(object):
    def __init__(self,money=1000, hand=0, points=0, play='in_play'):
        self.money = money
        self.hand = hand
        self.points = points
        self.play = play
    
    def addmoney(self,amount=100):
        self.money += amount
        
    def lessmoney(self,amount=100):
        self.money -= amount
        
    def draw(self,hand): #number of cards drawn
        self.hand += hand
        
    def addpoints(self,points):
        self.points += points
        
    def stand(self):
        self.play = 'stop'
        
    def bust(self):
        self.play = 'bust'
        
Player1 = Player()
Dealer = Player()

bet = 0
Gameplay = True

In [3]:
#creating the deck
decksuits = ['Diamonds', 'Clubs', 'Hearts', 'Spades']
decknum = (list(range(2,11)) + ['Jack','Queen','King','Ace'])
deck = []

for suit in decksuits:
    for num in decknum:
        deck.append((num,suit))

#shuffling the deck
np.random.shuffle(deck)
deck_iter = iter(deck)

In [4]:
#creating the deck points system
class Cards(object):
    def __init__(self,points):
        self.points = points

Picture = Cards(points=10) #Jack, Queen or King cards
Dealer_Ace = Cards(points=1 if Dealer.points > 10 else 11) #ace points for dealer

In [5]:
#ace points for player by choice
def acepoint():
    while True:
        try:
            choice = int(input("You have drawn an Ace. Do you want your Ace points to count as 11 or 1?"))
            return(choice)
        except:
            print("Please try again and only input 11 or 1.")
            continue
        else:
            break

In [6]:
#calculating points of card
def points(cardnum,turn):
    if type(cardnum) == int:
        return cardnum
    elif cardnum == 'Jack' or cardnum == 'Queen' or cardnum == 'King':
        return Picture.points
    elif cardnum == 'Ace' and turn == 'dealer':
        return Dealer_Ace.points
    elif cardnum == 'Ace' and turn == 'player':
        return acepoint()

In [7]:
#draw a card
def draw(turn):
    cardnum, cardsuit = next(deck_iter)
    point = points(cardnum,turn)
    if turn == 'player':
        Player1.draw(1)
        Player1.addpoints(point)
        print("You have drawn the " + str(cardnum) + " of " + str(cardsuit) + ". Your current points are " + str(Player1.points) + ".")
    else:
        Dealer.draw(1)
        Dealer.addpoints(point)
        print("Dealer drew the " + str(cardnum) + " of " + str(cardsuit) + ". His current points are " + str(Dealer.points) + ".")

In [8]:
#check money
def checkmoney():
    if Player1.money > 0:
        print("You currently have $" + str(Player1.money) + " worth of chips.")
    else:
        print("Unfortunately you have no more money left! Game Over.")
        global Gameplay
        Gameplay = False

In [9]:
#endgame
def endgame():
    checkmoney()
    global Gameplay
    while Gameplay == True:
        replay = input("Do you want to play again? Y/N: ")
        if replay[0].lower() == 'y':
            Player1.points = 0
            Dealer.points = 0
            Player1.hand = 0
            Dealer.hand = 0
            Player1.play = 'in_play'
            break
        elif replay[0].lower() == 'n':
            print("Game Over! Your final money pool is " + str(Player1.money) + ".")
            Gameplay = False
            break
        else:
            print("Try again - please input only Y/N.")
            continue

In [10]:
#check for immediate blackjack
def checkblackjack():
    if Player1.points == 21 and Player1.hand == 2 and Dealer.points == 21 and Dealer.hand == 2:
        print("It's a Tie! Your bet is returned.")
        endgame()
    elif Player1.points == 21 and Player1.hand == 2:
        print("BlackJack! You win double your bet amount.")
        Player1.addmoney(bet * 2)
        endgame()
    elif Dealer.points == 21 and Dealer.hand == 2:
        print("Dealer got a BlackJack! You lose double your bet amount.")
        Player1.lessmoney(bet * 2)
        endgame()
    else:
        pass

In [11]:
#check player decision
def decision():
    while Gameplay == True:
        decision = input("Player 1, would you like to Hit or Stand?")
        if decision[0].lower() == 'h':
            draw('player')
            break
        elif decision[0].lower() == 's':
            Player1.play = 'stop'
            break
        else:
            print("Try Again - please input Hit or Stand.")
            continue

In [12]:
#check for bust
def bust(turn):
    if turn == 'player':
        if Player1.points > 21:
            print("Busted - you lose!")
            Player1.lessmoney(bet)
            endgame()
        else:
            pass
    else:
        if Dealer.points > 21:
            print("Dealer Busted - you win!")
            Player1.addmoney(bet)
            endgame()
        else:
            pass

In [13]:
#check for win
def checkwin():
    if Player1.points > Dealer.points:
        print("You win!")
        Player1.addmoney(bet)
        endgame()
    elif Player1.points == Dealer.points:
        print("It's a Tie! Your bet is returned.")
        endgame()
    elif Player1.points < Dealer.points:
        print("You lose!")
        Player1.lessmoney(bet)
        endgame()

In [14]:
#dealer move
def dealer():
    while Dealer.points < 17:
        draw('dealer')
    else:
        pass

In [15]:
#ask for bet
def ask_bet():
    while True:
        try:
            bet_amount = int(input("How much would you like to bet?"))
            assert bet_amount < (Player1.money + 1) and bet_amount > 0
            global bet
            bet = bet_amount
            print("Your bet is " + str(bet) + ".")
        except:
            print("Try again - please only input whole numbers less than 1000.")
        else:
            break

In [16]:
#GamePlay

#Welcome
print("""
Welcome to BlackJack! 
You are playing against the Dealer, and you begin with $1000 worth of chips.
This game follows the standard rules of BlackJack.
""")

#start of loop
while Gameplay == True:
    ask_bet()
    draw('player') #player draws 2 cards
    draw('player')
    checkblackjack()
    print("You are able to see one of the Dealer's cards.")
    draw('dealer') #dealer draws 1 card

    while Player1.hand >= 2 and Gameplay == True: #to avoid loop starting here in new game
        if Player1.hand >= 2 and Player1.play == 'in_play':
            decision()
            bust('player')
        else:
            #dealer's turn and check for win
            dealer()
            bust('dealer')
            if Player1.hand >= 2 and Dealer.hand >= 2:
                checkwin()
                break
            break


Welcome to BlackJack! 
You are playing against the Dealer, and you begin with $1000 worth of chips.
This game follows the standard rules of BlackJack.

How much would you like to bet?1000
Your bet is 1000.
You have drawn the 5 of Diamonds. Your current points are 5.
You have drawn the 8 of Diamonds. Your current points are 13.
You are able to see one of the Dealer's cards.
Dealer drew the Ace of Hearts. His current points are 11.
Player 1, would you like to Hit or Stand?h
You have drawn the 6 of Hearts. Your current points are 19.
Player 1, would you like to Hit or Stand?h
You have drawn the 7 of Hearts. Your current points are 26.
Busted - you lose!
Unfortunately you have no more money left! Game Over.
